In [6]:
import numpy as np
import pandas as pd
import math


#PARAMETERS
T=5  #Number of iterations for a given run
N = 20 #Number of runs
DEBUG_PRINT = 1

#Use this to implement the "framework"


#User libraries
#from filename import class
from mab_base import mab_base
from collab_filter import MF
import thompson_algo as ta
import UCB as ucb
import Epsilon_Greedy as ep


#Python libaries
from math import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

#def DCG_compute():
#   
##   Implementing equation 3    
#def DCG_Normalize(self,n_users,dcg_true,dcg_pred):
#	for index in range(len(dcg_true)) :
#		ndcg += dcg_pred[index]/dcg_user[index]
#	return ndcg/n_users
#    
##   Implementing equation 4


def DCG_Normalize(dcg_pred,dcg_true):
    assert(len(dcg_true) == len(dcg_pred) )
    assert(len(dcg_true) > 0)
    ndcg = 0
    for index in range(len(dcg_true)) :
        if(dcg_true[index] != 0):
            ndcg += dcg_pred[index]/dcg_true[index]
        else:
            assert(1)
    print(dcg_true)
    return ndcg/len(dcg_true)


def DCG_Pred_perUser(r_u_t):
    dcg = r_u_t[0]
    for time in range(len(r_u_t)) :
        if (time > 1) :
            dcg += r_u_t[time]/(math.log(time,2)) 
    return dcg

def printClusters(mab):
    print('Current Cluster State:')
    print(mab.clusters)
    print('')
    print('Original/Safe Clusters:')
    print(mab.clusters_safe)

def thompsonState(mab):
    print('--- MAB Status -------------------------------------------')
    print('Arms:\t\t', end = '')
    print(mab.k)
    print('')
    print('Iterations:\t', end='')
    print(mab.n)
    print('')
    print('Arm Alpha:\t', end='')
    print(mab.k_a)
    print('')
    print('Arm Beta:\t', end='')
    print(mab.k_b)
    print('')
    print('Arm Theta:\t', end='')
    print(mab.k_theta)
    print('')
    print('Arm Reward:\t', end='')
    print(mab.k_reward)
    print('')
    printClusters(mab)
    print('----------------------------------------------------------')
    
def getRewards(X_test, rec_movie):  #get true value of current user and the recommended movie/or recommended cluster
    temp = X_test.iloc[:,rec_movie:rec_movie+1]
    reward = temp[rec_movie].values[0]/np.max(X_test)[0]    
    return reward

#STEP1: Read csvs and create nice DS out of them
	
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

# Merge the two files
ratings = pd.merge(movies,ratings,on='movieId')

# Creating the sparse matrix
userRatings = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating').fillna(0)
userRatings.head()

# Creating predicted rating matrix
R = userRatings.values

#Creating the Collab filtering object
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=1)

training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)
print("MSE:")
print(mf.mse())

final_predicted_matrix = pd.DataFrame(mf.full_matrix())
print(final_predicted_matrix)



P x Q:
[[5.09455269 3.99673945 4.27217546 ... 4.18336384 4.28571009 4.44745598]
 [4.61401169 3.62448672 3.59493399 ... 3.76886657 3.12013569 3.67650745]
 [2.68993373 2.51248678 2.67247747 ... 2.96800982 3.42900402 2.87830047]
 ...
 [4.03278836 2.89052776 3.10985281 ... 3.04852638 2.96562811 3.26306127]
 [3.06467141 2.49066623 2.83243873 ... 2.8638061  3.52971328 3.06919013]
 [4.10236214 3.36216791 3.42110072 ... 3.60975723 3.38420064 3.55093626]]

Global bias:
3.501556983616962

User bias:
[ 8.54765751e-01  1.20736108e-01 -6.42037722e-01  4.00935406e-02
  1.70444169e-01  2.15053160e-01 -2.48893737e-01  2.39009089e-01
 -5.19106221e-01 -9.01427149e-01  3.99225340e-01  1.08795312e+00
  1.30095264e-01 -9.19441732e-02 -5.38746111e-01 -1.30761213e-01
  7.29928721e-02 -1.14105678e-01 -1.08963094e+00  1.07754819e-01
 -6.40622939e-01 -1.69603757e+00 -2.75470252e-01 -8.03928645e-02
  3.98626476e-01 -2.87031316e-01  3.44868124e-01 -5.63516394e-01
  2.01526038e-01  4.83540602e-01  4.74831446e-01 

275.36088325332526
         0         1         2         3         4         5         6     \
0    5.094553  3.996739  4.272175  3.650015  4.038414  4.911708  4.091306   
1    4.614012  3.624487  3.594934  3.124452  2.635617  3.743007  2.921325   
2    2.689934  2.512487  2.672477  2.488465  2.459686  3.509835  2.404203   
3    4.909881  3.276566  3.536837  2.688038  3.085174  3.915831  3.270507   
4    4.690838  3.451258  3.630621  2.980092  3.109183  4.038569  3.274715   
5    4.768064  3.246223  3.659786  2.806580  3.628571  4.358214  3.659814   
6    3.867549  2.932059  3.157399  2.622731  2.853048  3.779297  2.911884   
7    4.427645  3.484109  3.658488  3.137846  3.224116  4.186857  3.327695   
8    3.982983  2.496800  2.918629  2.078020  2.920051  3.647222  2.936361   
9    4.006374  2.461134  2.609350  1.837186  1.907346  2.828857  2.167427   
10   4.655352  3.803594  3.838659  3.405621  3.087089  4.161008  3.287559   
11   5.048655  3.925820  4.450076  3.731588  4.829534  5.

In [7]:
dcg_thom=[]
dcg_ucb = []
dcg_greedy = []
dcg_thom_true=[]
dcg_ucb_true = []
dcg_greedy_true = []

In [ ]:
for i in range(0,N):
    #Add cluster classification here
    # Creating Clusters
    print(i)
    #LOO here 
    X_train, X_test = train_test_split(final_predicted_matrix, test_size=0.001)
    print(X_test)


    sse = []
    #list_k = list(range(4, 20))

    #for k in list_k:
    #km = KMeans(n_clusters=5,n_init = 20, n_jobs=-1)
    #km.fit(X_train)
    #sse.append(km.inertia_)

    # Plot sse against k
    #plt.figure(figsize=(6, 6))
    #plt.plot(list_k, sse, '-o')
    #plt.xlabel(r'Number of clusters *k*')
    #plt.ylabel('Sum of squared distance');


    model_knn = KMeans(n_clusters=3, n_init = 20, random_state = 42, n_jobs=-1)
    model_knn.fit(X_train)

    # new cluster labels

    y_km = model_knn.fit_predict(X_train)
    final_clusters = model_knn.cluster_centers_

    # Final Clusters
    final_clusters = pd.DataFrame(final_clusters)

    # Creating a new initial column with cluster name
    new_col = {'cluster': [i for i in range(1,len(final_clusters)+1)]}
    new_col = pd.DataFrame(new_col)
    #Setting cluster for other algorithms to use
    final_clusters = new_col.join(final_clusters)
    print("Done")

    # create thompson mab object
    r_u_t_thom  = []
    r_u_t_thom_true  = []

    mab_0 = ta.thompson()    
    mab_0.set_clusters(final_clusters)
    #thompsonState(mab_0)
    last_reward = 0
    for i in range(0,T):
        # pull arm
        #print('Pull ',i)
        print("Movie Recommended:\t", end='')
        rec_movie_id = mab_0.pull_arm(last_reward)
        print(rec_movie_id)
        #mab_0.pull_arm(last_reward)
        #thompsonState(mab_0)
        print('')
        last_reward = getRewards(X_test,rec_movie_id) 
        #print("last_reward: ", last_reward)
        temp = X_test.iloc[:,rec_movie_id:rec_movie_id+1]
        #print(temp[rec_movie_id].values[0])
        r_u_t_thom.append(temp[rec_movie_id].values[0])
        #r_u_t_thom_true.append( X_test.max().values[0] )
        r_u_t_thom_true.append(np.max(X_test,axis =1).values[0])


    dcg_thom.append(DCG_Pred_perUser(r_u_t_thom))
    
    dcg_thom_true.append(DCG_Pred_perUser(r_u_t_thom_true))
    print("True DCG for current user for Thompson: ", dcg_thom_true)
    print("DCG for current user for Thompson: ", dcg_thom)


    # reset
    mab_0.reset()
    #thompsonState(mab_0)
    print('')

    # create ucb mab object
    r_u_t_ucb = []
    r_u_t_ucb_true = []
    mab_0 = ucb.ucb_bandit()
    mab_0.set_clusters(final_clusters)
    #thompsonState(mab_0)
    last_reward = 0
    for i in range(0,T):
        # pull arm
        #print('Pull ',i)
        #print("Movie Recommended:\t", end='')
        rec_movie_id = mab_0.pull_arm(last_reward)
        #mab_0.pull_arm(last_reward)
        #print('')
        last_reward = getRewards(X_test,rec_movie_id) 
        #print("last_reward: ", last_reward)
        temp = X_test.iloc[:,rec_movie_id:rec_movie_id+1]
        #print(temp[rec_movie_id].values[0])
        r_u_t_ucb.append(temp[rec_movie_id].values[0])
        #r_u_t_ucb_true.append( X_test.max().values[0] )
        r_u_t_ucb_true.append(np.max(X_test,axis =1).values[0])



    dcg_ucb.append(DCG_Pred_perUser(r_u_t_ucb))
    dcg_ucb_true.append(DCG_Pred_perUser(r_u_t_ucb_true))
    print("True DCG for current user for UCB: ", dcg_ucb_true)
    print("DCG for current user for UCB: ", dcg_ucb)

    # reset

    mab_0.reset()
    print('')

    #Greedy object
    r_u_t_greedy = []
    r_u_t_greedy_true = []

    k = final_clusters.shape[0]
    mab_0 = ep.eps_bandit()
    mab_0.set_clusters(final_clusters)
    last_reward = 0
    for i in range(0,T):
        # pull arm
        #print('Pull ',i)
        #print("Movie Recommended:\t",end='')
        rec_movie_id = mab_0.pull_arm(last_reward)
        #mab_0.pull_arm(last_reward)
        #thompsonState(mab_0)
        print('')
        last_reward = getRewards(X_test,rec_movie_id)
        #print("last_reward: ", last_reward)
        temp = X_test.iloc[:,rec_movie_id:rec_movie_id+1]
        #print(temp[rec_movie_id].values[0])
        r_u_t_greedy.append(temp[rec_movie_id].values[0])
        #r_u_t_greedy_true.append( X_test.max().values[0] )
        r_u_t_greedy_true.append( (np.max(X_test,axis =1).values[0]))


    dcg_greedy.append(DCG_Pred_perUser(r_u_t_greedy))
    dcg_greedy_true.append(DCG_Pred_perUser(r_u_t_greedy_true))
    print("True DCG for current user for Greedy: ", dcg_greedy_true)
    print("DCG for current user for Greedy: ", dcg_greedy)

    # reset
    mab_0.reset()
    print('')

0
        0         1         2         3         4        5         6     \
276  4.00554  3.656973  3.398199  3.283985  2.081676  3.42116  2.417325   

         7         8         9     ...      9714      9715     9716      9717  \
276  3.158414  2.544676  3.251503  ...  3.100184  3.735007  3.53078  2.946379   

         9718      9719     9720      9721     9722     9723  
276  3.452221  3.570914  3.38627  3.950886  3.14679  3.45135  

[1 rows x 9724 columns]
Done
Movie Recommended:	277

Movie Recommended:	1938

Movie Recommended:	3617

Movie Recommended:	960

Movie Recommended:	3633

True DCG for current user for Thompson:  [24.50621725386986]
DCG for current user for Thompson:  [13.741647319650124]

0
1
2
2
1
True DCG for current user for UCB:  [24.50621725386986]
DCG for current user for UCB:  [11.190951528800962]






True DCG for current user for Greedy:  [24.50621725386986]
DCG for current user for Greedy:  [13.44464510226478]

1
        0         1         2         3       

         0         1         2        3         4         5         6     \
535  4.286536  3.400117  3.544249  3.05873  3.053509  4.044121  3.169814   

         7         8         9     ...      9714      9715      9716  \
535  3.320613  3.129245  3.724827  ...  3.621413  3.620126  3.808755   

         9717      9718      9719      9720      9721      9722      9723  
535  3.656677  3.591689  3.725469  3.595147  3.621936  3.521399  3.691555  

[1 rows x 9724 columns]
Done
Movie Recommended:	277

Movie Recommended:	3617

Movie Recommended:	1257

Movie Recommended:	1938

Movie Recommended:	960

True DCG for current user for Thompson:  [24.50621725386986, 16.279328528913076, 13.14700666794589, 17.680623887987018, 16.32863191250633, 13.777853189225032, 14.301272387068366, 15.215796730051743]
DCG for current user for Thompson:  [13.741647319650124, 15.043865055320095, 9.838148586219775, 14.254620111428096, 15.069907764082041, 12.979691367514702, 13.916613820386054, 14.581379993702592]

0

In [ ]:
ndg_greedy = DCG_Normalize(dcg_greedy,dcg_greedy_true)
ndg_ucb = DCG_Normalize(dcg_ucb,dcg_ucb_true)
ndg_thom = DCG_Normalize(dcg_thom,dcg_thom_true)

print("NDCG for Thompson Algorithm for ",N, "users over time ",T, ": ", ndg_thom)
print("NDCG for UCB Algorithm for ",N, "users over time ",T, ": ", ndg_ucb)
print("NDCG for Greedy Algorithm for ",N, "users over time ",T, ": ", ndg_greedy)

